In [1]:
import pandas as pd

# Read the first file (CSV)
input_file_1_df = pd.read_csv('C:/Users/algo/Desktop/VS Code/Task/Philly Deals to email append._processed.csv')

# Read the second file (Excel)
input_file_2_df = pd.read_excel('C:/Users/algo/Desktop/VS Code/Task/sql_phoenix_20230921_141556.xlsx')


# Define a function to find the first empty email column in file2
def find_empty_email_column(matching_row):
    for i in range(2, 6):
        email_column = f"Email{i}"
        if pd.isna(matching_row[email_column]):
            return email_column
    return None

# Define a function to clean a column and make it a unique key
def clean_and_make_unique_key(df, column_name):
    if column_name in df.columns:
        if df[column_name].dtype == 'object':  # Check if the column contains string values
            df[column_name] = df[column_name].str.strip()  # Remove leading/trailing spaces
            df[column_name] = df[column_name].str.upper()  # Convert to uppercase for case-insensitive comparison
        df[column_name] = df[column_name].fillna('')    # Replace NaN values with empty string
        df[column_name] = df[column_name].drop_duplicates(keep='first')
    return df


# Clean and make unique keys for both DataFrames
composite_key = ["Business Name", "Address", "City", "State", "Postal Code", "Phone"]
for column in composite_key:
    input_file_1_df = clean_and_make_unique_key(input_file_1_df, column)
    input_file_2_df = clean_and_make_unique_key(input_file_2_df, column)

# Define a function to create a composite key from multiple columns
def create_composite_key(row):
    key_values = [str(row[column]) for column in composite_key if column in row.index]
    return '|'.join(key_values)


# Create the composite key column in both DataFrames
input_file_1_df['Composite Key'] = input_file_1_df.apply(create_composite_key, axis=1)
input_file_2_df['Composite Key'] = input_file_2_df.apply(create_composite_key, axis=1)

# Iterate through the rows in file1 and update corresponding rows in file2
for _, row in input_file_1_df.iterrows():
    composite_key_value = row['Composite Key']
    
    # Create a mask to filter rows in file2 that match the composite key
    mask = (input_file_2_df['Composite Key'] == composite_key_value)
    
    # Check if a matching row exists in file2
    matching_rows = input_file_2_df[mask]
    if not matching_rows.empty:
        # Check if the email in file1 is different from the emails in file2
        if row["Email"] != matching_rows.iloc[0]["Email"]:
            # Find the first empty email column in file2
            empty_email_column = find_empty_email_column(matching_rows.iloc[0])
            if empty_email_column:
                # Move existing email data to the next available email column in file2
                for i in range(5, int(empty_email_column[-1]), -1):
                    input_file_2_df.at[matching_rows.index[0], f"Email{i}"] = input_file_2_df.at[matching_rows.index[0], f"Email{i-1}"]
                # Fill the empty email column in file2 with the email from file1
                input_file_2_df.at[matching_rows.index[0], empty_email_column] = row["Email"]
# Save the updated file2 DataFrame back to an Excel file
input_file_2_df.to_excel('output.xlsx', index=False)


C:\Users\algo\AppData\Local\Temp\ipykernel_9464\2054399671.py:4: DtypeWarning: Columns (25) have mixed types. Specify dtype option on import or set low_memory=False.
  input_file_1_df = pd.read_csv('C:/Users/algo/Desktop/VS Code/Task/Philly Deals to email append._processed.csv')
